In [93]:
import base64
import hashlib
import html
import json
import os
import re
import urllib.parse
import requests

In [94]:
provider = "http://localhost:8080/realms/fetchusers"
client_id = "ch"
username = "123"
password = "123"
redirect_uri = "*"


In [95]:
code_verifier = base64.urlsafe_b64encode(os.urandom(40)).decode('utf-8')
code_verifier = re.sub('[^a-zA-Z0-9]+', '', code_verifier)
code_verifier, len(code_verifier)

('dcNOJE33XvxjvXLQZBpkcZPIsZulZEXpH2xlwEgHgJRjhnDjGcQ', 51)

In [96]:
code_challenge = hashlib.sha256(code_verifier.encode('utf-8')).digest()
code_challenge = base64.urlsafe_b64encode(code_challenge).decode('utf-8')
code_challenge = code_challenge.replace('=', '')
code_challenge, len(code_challenge)

('GmqV7p3ylsSblNVuI1HqJ3fUVi9iXVUulGj7Gq0Mzwc', 43)

In [97]:
state = "fooobarbaz"
resp = requests.get(
    url=provider + "/protocol/openid-connect/auth",
    params={
        "response_type": "code",
        "client_id": client_id,
        "scope": "openid",
        "redirect_uri": redirect_uri,
        "state": state,
        "code_challenge": code_challenge,
        "code_challenge_method": "S256",
    },
    allow_redirects=False
)
resp.status_code

200

In [98]:
cookie = resp.headers['Set-Cookie']
cookie = '; '.join(c.split(';')[0] for c in cookie.split(', '))
cookie

'AUTH_SESSION_ID=3bc142dd-8ad6-44f4-8c78-076ac92a47f1; AUTH_SESSION_ID_LEGACY=3bc142dd-8ad6-44f4-8c78-076ac92a47f1; KC_RESTART=eyJhbGciOiJkaXIiLCJlbmMiOiJBMTI4Q0JDLUhTMjU2In0..d3PX64HIln6JIjf6gLsvZg.S2omXQGpYVTKulzF2uhrZ5k11mBqG7Hc7fe6xZ6gnEFGba34pqTu-gA6yMiPcQEcHSAh1PHzTu6s9j8OGyAfhtZZcj2bK4bW0Oyv3UeRqatFUZ7iXwtNh_e0MdiD2RvB5oirxJUQtmPUgdyuE8QGyRu8g43sG83Bv8DpSUybcpOpM4tUO3MxTBIBA531fPpTQnUu7Tdy8vdpjeRkqOG5beNrpcz4j9pxPx7SeGp05qN_hSePx1WiWTdPfPuiBqr8dH4tCUGA6CrIRZ35xTwRtAv3Jh5vX-G3_7fO1w1a-uqqt1ldVxcUSmsCu3y0z6P3iU5QXQxCsITKqx52lI8nHD0_1c5J6U1sidasqdKiapOcCRsHScEHn8f5tQZA9csR4S3GWRtgnJgPF_v_VqcicW7dHnBmwR5eZYyE05WATabfmFaRTN8q_z3wqwrvsb_JClLwEZ1c0dIiiP-LUME57KXMSOHMO4g5fWxp_wzwod461xPX90suBF3q6LnxI_hwwvcI1fmgcAhesSiqEksfmIBBzBPPLnCf6urKH-6oaVFq8c7JqywzFOtrLjyfWv9fHuAkcglJzsbtzVBlg6JCrHr-xqtmy61-8uCJm9JwalEGtB3zrkE-2AlUgYizgiT2ydQA7vO4s98Jku94iru3-I0QvSU8iC4RZUdctfxcks1qZ1v0VfoSaK1gy44YyLZXLeZnvQxn0rlXQ0fE4KPT73cDwqkzXagBWbBvUKRCAPBWUHNl9lS1R4lIAzVhC5d8aXDA3_EF-FUWU9VzoXUhpwwAjH5vkxANA

In [100]:
page = resp.text
form_action = html.unescape(re.search(r'<form\s+.*?\s+action="(.*?)"', page, re.DOTALL).group(1))
form_action

'http://localhost:8080/realms/fetchusers/login-actions/authenticate?session_code=HUHHTJeB8aPsjdPZVM_dt7Hbld32IUuSqutpSOgCmmA&execution=2d96f996-d368-4e10-bdeb-8b877788645e&client_id=ch&tab_id=M97ogabatW4&client_data=eyJydSI6Imh0dHA6Ly9sb2NhbGhvc3Q6ODA4MC8qIiwicnQiOiJjb2RlIiwic3QiOiJmb29vYmFyYmF6In0'